In [1]:
import pandas as pd
import plotly.graph_objects as go
import context
from evento import ajustar_datas

In [2]:
# Fundo sem histórico completo de preços no Yahoo Finance
# Carregar histórico da B3
precos = (
    pd.read_csv('data/krni.csv', parse_dates=['datneg'])
    # a série da B3 começa em "2010-12-01" -> vamos começar em "2011-01-01"
    .query('datneg >= "2011-01-01"')
)
precos

,datneg,preabe,premed,preult
20,2011-01-04,100.10,100.05,100.00
21,2011-01-05,100.10,99.99,99.70
22,2011-01-06,99.70,99.70,99.70
23,2011-01-07,99.50,99.50,99.50
24,2011-01-10,100.00,97.30,95.50
...,...,...,...,...
2984,2022-12-26,140.94,139.20,138.60
2985,2022-12-27,139.20,138.55,137.80
2986,2022-12-28,137.90,139.50,140.98
2987,2022-12-29,140.98,140.54,140.20


In [3]:
# Histórico de dividendos do HGLG11 extraído do site Funds Explorer
# Praticamente não há dados de dividendos para o HGLH11 no Yahoo Finance
divs = (
    pd.read_csv('data/krni_divs.csv', parse_dates=['data_com', 'data_pag'])
    # a série da B3 começa em "2010-12-01" -> vamos começar em "2011-01-01"
    .query('data_com >= "2011-01-01"')

)
divs

,tipo,data_com,data_pag,valor_div
1,Rendimento,2011-01-31,2011-02-14,0.675
2,Rendimento,2011-02-28,2011-03-16,0.700
3,Rendimento,2011-03-31,2011-04-14,0.700
4,Rendimento,2011-04-29,2011-05-13,0.700
5,Rendimento,2011-05-31,2011-06-14,0.700
...,...,...,...,...
139,Rendimento,2022-07-29,2022-08-12,0.910
140,Rendimento,2022-08-31,2022-09-15,0.910
141,Rendimento,2022-09-30,2022-10-17,0.910
142,Rendimento,2022-10-31,2022-11-16,0.910


In [5]:
# Assegurar que as datas dos eventos sejam em datas com negociação do ativo
# anterior=True: se não houver negociação na data_com, ajustar para a primeira data anterior de negociação
divs['data_com'] = ajustar_datas(divs['data_com'], precos['datneg'])
# anterior=False: se não houver negociação na data de pagamento, ajustar para a primeira data posterior
divs['data_pag'] = ajustar_datas(divs['data_pag'], precos['datneg'], anterior=False)
divs

,tipo,data_com,data_pag,valor_div
1,Rendimento,2011-01-31,2011-02-14,0.675
2,Rendimento,2011-02-28,2011-03-16,0.700
3,Rendimento,2011-03-31,2011-04-14,0.700
4,Rendimento,2011-04-29,2011-05-13,0.700
5,Rendimento,2011-05-31,2011-06-14,0.700
...,...,...,...,...
139,Rendimento,2022-07-29,2022-08-12,0.910
140,Rendimento,2022-08-31,2022-09-15,0.910
141,Rendimento,2022-09-30,2022-10-17,0.910
142,Rendimento,2022-10-31,2022-11-16,0.910


In [10]:
# Nehuma data_com foi ajustada
set(divs['data_com']) - set(ajustar_datas(divs['data_com'], precos['datneg']))

set()

In [8]:
precos.query('datneg >= "2011-01-25"')

,datneg,preabe,premed,preult
34,2011-01-26,97.80,97.38,97.50
35,2011-01-27,97.50,97.07,97.00
36,2011-01-28,97.00,97.00,96.60
37,2011-01-31,96.60,96.12,96.99
38,2011-02-01,96.99,96.99,96.99
...,...,...,...,...
2984,2022-12-26,140.94,139.20,138.60
2985,2022-12-27,139.20,138.55,137.80
2986,2022-12-28,137.90,139.50,140.98
2987,2022-12-29,140.98,140.54,140.20


In [5]:
# Calcular o dividend_yield
precos["dividend_yield(%)"] = 100 * precos["valor_div"] / precos["preco"]
precos.query("valor_div > 0")

,data,preco,preco_aj,valor_div,dividend_yield(%)
19,2011-03-31,104.401001,37.354568,0.72,0.689649
38,2011-04-29,104.400002,37.613612,0.75,0.718391
60,2011-05-31,104.300003,37.849491,0.79,0.757430
81,2011-06-30,106.300003,38.869683,0.80,0.752587
102,2011-07-29,109.000000,40.159198,0.81,0.743119
...,...,...,...,...,...
2831,2022-07-29,168.639999,163.207106,1.10,0.652277
2854,2022-08-31,172.490005,168.029096,1.10,0.637718
2875,2022-09-30,171.500000,168.136933,1.10,0.641399
2895,2022-10-31,165.389999,163.193470,1.10,0.665095


In [6]:
precos.query('data >= "2022-11-27"')

,data,preco,preco_aj,valor_div,dividend_yield(%)
2913,2022-11-28,162.899994,161.812742,NaN,NaN
2914,2022-11-29,163.500000,162.408743,NaN,NaN
2915,2022-11-30,164.809998,163.709998,1.1,0.667435
2916,2022-12-01,162.710007,162.710007,NaN,NaN
2917,2022-12-02,162.800003,162.800003,NaN,NaN
2918,2022-12-05,162.839996,162.839996,NaN,NaN
2919,2022-12-06,162.949997,162.949997,NaN,NaN
2920,2022-12-07,162.179993,162.179993,NaN,NaN
2921,2022-12-08,162.199997,162.199997,NaN,NaN
2922,2022-12-09,162.970001,162.970001,NaN,NaN


In [7]:
# Valorização do HGLG11 com e sem ajuste de dividendos
valorizacao_sa = (precos["preco"].iloc[-1] / precos["preco"].iloc[0] - 1) * 100
valorizacao_ca = (precos["preco_aj"].iloc[-1] / precos["preco_aj"].iloc[0] - 1) * 100
print(f"Valorização do HGLG11 com ajuste de dividendos: {valorizacao_sa:.2f}%")
print(f"Valorização do HGLG11 sem ajuste de dividendos: {valorizacao_ca:.2f}%")

Valorização do HGLG11 com ajuste de dividendos: 54.59%
Valorização do HGLG11 sem ajuste de dividendos: 332.06%


In [8]:
# Verificar o histórico de dividendos da TAEE11
# Na comparação com o site da B3, parece estar tudo OK
# Se o histórico de dividendos estiver completo -> preço ajustado está Ok também
# Carregar o histórico de dividendos do TAEE11 com o yfinance
taee_divs = yf.Ticker("TAEE11.SA").dividends
taee_divs = taee_divs.reset_index()
taee_divs.columns = ["data", "valor_div"]
# Remover o timezone da data
taee_divs["data"] = taee_divs["data"].dt.tz_localize(None)
dividendos_12m = taee_divs.query('data >= "2021-12-15"')["valor_div"].sum()
print(f"Dividendos pagos nos últimos 12 meses: R$ {dividendos_12m:.2f}")
print(f"Valor esperado dos últimos 12 meses = R$ 4,85")
print(f"DADOS DOS YAHOO FINANCE ESTÃO INCOMPLETOS!!!")
taee_divs.tail(5)

Dividendos pagos nos últimos 12 meses: R$ 3.50
Valor esperado dos últimos 12 meses = R$ 4,85
DADOS DOS YAHOO FINANCE ESTÃO INCOMPLETOS!!!


,data,valor_div
39,2021-05-19,1.354363
40,2021-12-07,1.518146
41,2022-05-10,1.896324
42,2022-08-16,1.470929
43,2022-11-16,0.131087


In [10]:
# Carregar CSV com os dividendos da TAEE11
# Os dados foram extraídos do site da B3 e as datas são "data-com"
taee_divs = pd.read_csv('data/taee_dividendos.csv', parse_dates=['data_com'])
dividendos_12m = taee_divs.query('data_com >= "2021-12-15"')["valor_div"].sum()
print(f"Dividendos pagos nos últimos 12 meses: R$ {dividendos_12m:.2f}")
taee_divs

Dividendos pagos nos últimos 12 meses: R$ 4.85


,data_com,valor_div
0,2011-04-15,2.810692
1,2012-04-27,1.626325
2,2012-12-18,0.464443
3,2013-04-30,1.042921
4,2013-12-12,0.609581
...,...,...
57,2022-05-09,2.323063
58,2022-08-15,0.896372
59,2022-08-15,0.574557
60,2022-11-14,0.460260


In [10]:
# Fazer um gráfico de barras com o dividendos pagos pela TAEE11 a cada ano usano o plotly

# Calcular o ano de cada data-com
taee_divs["ano"] = taee_divs["data_com"].dt.year
# Agrupar os dividendos por ano
taee_divs_gb = taee_divs.groupby("ano")["valor_div"].sum().reset_index()
# Criar o gráfico
fig = go.Figure(
    data=[
        go.Bar(
            x=taee_divs_gb["ano"],
            y=taee_divs_gb["valor_div"],
            text=taee_divs_gb["valor_div"].round(1),
            textposition="auto",
        )
    ]
)
fig.update_layout(
    title="Dividendos pagos pela TAEE11 a cada ano",
    xaxis_title="Ano",
    yaxis_title="Dividendos (R$)",
)
fig.show()

In [10]:
# Histórico dos preços de fechamento do TAEE11
taee_precos = yf.download("TAEE11.SA", start="2011-03-02")[["Close", "Adj Close"]]
taee_precos = taee_precos.reset_index()
taee_precos.columns = ['data', 'preco', 'preco_aj_yf']
taee_precos

[*********************100%***********************]  1 of 1 completed


,data,preco,preco_aj_yf
0,2011-03-02,10.370000,3.276795
1,2011-03-03,10.370000,3.276795
2,2011-03-04,10.370000,3.276795
3,2011-03-09,10.370000,3.276795
4,2011-03-10,10.666666,3.370539
...,...,...,...
2932,2022-12-23,34.810001,34.810001
2933,2022-12-26,34.630001,34.630001
2934,2022-12-27,34.270000,34.270000
2935,2022-12-28,34.790001,34.790001


In [11]:
taee_precos = ajustar_precos(taee_precos, taee_divs)
taee_precos.query("valor_div > 0").tail(5)

,data,preco,preco_aj_yf,preco_aj,valor_div
2531,2021-05-18,40.700001,34.841217,33.697305,1.354364
2669,2021-12-06,37.990002,33.640778,32.536280,1.518147
2773,2022-05-09,45.009998,41.516159,40.153096,2.323063
2842,2022-08-15,43.380001,41.772614,40.805023,1.470929
2904,2022-11-14,40.259998,40.128910,39.199392,1.060606


In [12]:
# Calcular o dividend_yield
taee_precos["dividend_yield(%)"] = 100 * taee_precos["valor_div"] / taee_precos["preco"]
taee_precos.query("valor_div > 0")

,data,preco,preco_aj_yf,preco_aj,valor_div,dividend_yield(%)
30,2011-04-15,12.666666,4.002516,2.886661,2.810692,22.189676
287,2012-04-27,22.000000,7.723254,6.443458,1.626325,7.392386
446,2012-12-18,21.510000,7.841058,6.802836,0.464443,2.159194
533,2013-04-30,23.020000,8.576686,7.441062,1.042921,4.530499
690,2013-12-12,19.230000,7.504622,6.510948,1.161107,6.037997
781,2014-04-30,19.950001,8.015494,7.188786,1.200777,6.018930
884,2014-09-26,19.670000,8.409136,7.541828,1.161107,5.902932
932,2014-12-04,18.850000,8.058578,7.680819,0.571845,3.033661
1023,2015-04-22,21.010000,9.263009,8.828791,0.739500,3.519755
1125,2015-09-17,19.420000,8.874355,8.458357,0.899979,4.634289


In [13]:
# Adicionar uma coluna com o preço normalizado em taee_precos e precos
taee_precos["preco_normalizado"] = 100 * taee_precos["preco"] / taee_precos["preco"].iloc[0]
precos["preco_normalizado"] = 100 * precos["preco"] / precos["preco"].iloc[0]

In [16]:
# Plotar os preços da TAEE11 e HGLG11 sem ajustes
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=taee_precos["data"],
        y=taee_precos["preco_normalizado"],
        mode="lines",
        name="TAEE11",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=precos["data"],
        y=precos["preco_normalizado"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
    )
)
# Adicionar os pontos de dividendos da TAEE11 no próprio gráfico de preços
fig.add_trace(
    go.Scatter(
        x=taee_precos.query('valor_div > 0').data,
        y=taee_precos.query('valor_div > 0')["preco_normalizado"],
        mode="markers",
        name="Dividendos da TAEE11",
        marker=dict(
            color="black",
            size=(20 * taee_precos.query('valor_div > 0')["dividend_yield(%)"]) ** 0.5,
            symbol="circle"),
    )
)
# Adicionar os pontos de dividendos da HGLG11 no próprio gráfico de preços, sendo que o tamanho dos pontos é proporcional ao valor do dividendo
fig.add_trace(
    go.Scatter(
        x=precos.query('valor_div > 0').data,
        y=precos.query('valor_div > 0')["preco_normalizado"],
        mode="markers",
        name="Dividendos do HGLG11",
        marker=dict(
            color="black",
            size=(20 * precos.query('valor_div > 0')["dividend_yield(%)"]) ** 0.5,
            symbol="circle"),
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br> (sem ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()

In [15]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=taee_precos["data"],
        y=taee_precos["preco_aj"],
        mode="lines",
        name="TAEE11",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=precos["data"],
        y=precos["preco_aj"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
        
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br> (normalizados e com ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()

In [18]:
# Adicionar uma coluna com o preço ajustado e normalizado em taee_precos e precos
taee_precos["preco_aj_normalizado"] = 100 * taee_precos["preco_aj"] / taee_precos["preco_aj"].iloc[0]
precos["preco_aj_normalizado"] = 100 * precos["preco_aj"] / precos["preco_aj"].iloc[0]

In [20]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=taee_precos["data"],
        y=taee_precos["preco_aj_normalizado"],
        mode="lines",
        name="TAEE11",
        line=dict(color="#e86f00"),
    )
)
fig.add_trace(
    go.Scatter(
        x=precos["data"],
        y=precos["preco_aj_normalizado"],
        mode="lines",
        name="HGLG11",
        line=dict(color="#02878e"),
        
    )
)
fig.update_layout(
    font=dict(family="Fira Code", size=11, color="black"),
    title="Preços de fechamento <br> (com ajuste de dividendos)",
    title_x=0.5,
    title_y=0.85,
    xaxis_title="Data",
    yaxis_title="Preço (R$)",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()